In [ ]:
import requests
import csv
from bs4 import BeautifulSoup
import re

def clean_text(tag):
    return tag.get_text(strip=True) if tag else ""

# Fetch page
url = "https://www.punjabgrill.in/punjab-grill-menu/"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "lxml")

# Manual metadata fallback
restaurant_info = {
    "name": "Punjab Grill",
    "phone": "+91 9513720004",
    "address": "UB City, Bangalore",
    "opening_hours": "12:00 PM – 11:00 PM (Mon-Sun)"
}

def extract_menu_items():
    menu = []
    headings = soup.find_all("h3", class_="appetizer-list-heading")

    for heading in headings:
        section = clean_text(heading)
        section_block = heading.find_parent().find_next("div", class_="tab-content")
        if not section_block:
            continue

        tab_panes = section_block.find_all("div", class_="tab-pane")
        for pane in tab_panes:
            veg = "Veg" if "tab1" in pane.get("id", "").lower() else "Non-Veg"
            items = pane.select("div.panel-body ul.menu-list li")

            for item in items:
                text = clean_text(item)
                if not text:
                    continue

                # Pattern: everything before price (item), price, and everything after (desc)
                match = re.match(r'^(.*?)(\d{2,4})([^\d]*)$', text)
                if match:
                    item_name = match.group(1).strip()
                    price = int(match.group(2))
                    description = match.group(3).strip()
                else:
                    item_name = text
                    price = None
                    description = ""

                menu.append({
                    "section": section,
                    "type": veg,
                    "item": item_name,
                    "price": price,
                    "description": description
                })

    return menu

# Run and save
menu_items = extract_menu_items()

# Print summary
print("Restaurant:", restaurant_info['name'])
print("Phone:", restaurant_info['phone'])
print("Location:", restaurant_info['address'])
print("Opening Hours:", restaurant_info['opening_hours'])
print(f"\nTotal menu items scraped: {len(menu_items)}")

# Save to CSV
csv_file = "Punjab_grill_menu.csv"
with open(csv_file, "w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["section", "type", "item", "price", "description"])
    writer.writeheader()
    for row in menu_items:
        writer.writerow(row)

print(f"\nSaved to '{csv_file}'")


Restaurant: Punjab Grill
Phone: +91 9513720004
Location: UB City, Bangalore
Opening Hours: 12:00 PM – 11:00 PM (Mon-Sun)

Total menu items scraped: 62

Saved to 'Punjab_grill_menu.csv'
